# 18.335 Problem Set 1

This notebook accompanies the first problem set posted on the [18.335 web page](https://github.com/mitmath/18335), and is here to get you started with the Julia computations.

Download this notebook (a `pset1.ipynb` file) by **right-clicking the download link** at the upper-right to *Save As* a file, and then drag this file into your Jupyter dashboard to upload it (e.g. on [juliabox.com](https://juliabox.com/) or in a local installation).

Modify it as needed, then choose **Print Preview** from the "File" menu and *print to a PDF* file to submit electronically.

# Problem 1: Floating point

Give your solution to Trefethen problem 13.2(c) below.  Note that you can use the *Insert* menu (or ctrl-m b) to insert new code cells as needed.

In [9]:
import numpy as np 

B=2.0
p_single = 24.0
p_double = 53.0

n_single = B**p_single + 1

n_double = B**p_double + 1

print(n_single)
print(n_double)
x = [1,2,3]
for i in x: 
    print(f"x + {i}")
    print(n_double+i  == n_double)
    print(n_single+i == n_single)
    print(f"x - {i}")
    print(n_double - i  == n_double)
    print(n_single - i == n_single)



16777217.0
9007199254740992.0
x + 1
True
False
x - 1
False
False
x + 2
False
False
x - 2
False
False
x + 3
False
False
x - 3
False
False


# Problem 2: Funny functions

## part (a)

Compute $(|x|^4 + |y|^4)^{1/4}$:

In [26]:
def funny_1(x,y):
    M = max(np.abs(x),np.abs(y))
    return M*(np.abs(x/M)**4 + np.abs(y/M)**4)**(1/4)

Some tests:

In [27]:
x = np.arange(-500,500,1)

import matplotlib.pyplot as plt
plt.plot(x, funny_1(x,x))
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [28]:
(1e-100<-B**p_single - 1)

False

In [29]:
y = 0.0 
x = 1e-100 
funny_1(x,y)

1e-100

In [30]:
funny_1(1e+100,0.0)

1e+100

## part (b)

Compute $\cot(x) - \cot(x + y)$:
$$
\cot(x) - \cot(x+y) = \left[\cot(x+y) \cot(x) + 1\right] \tan(y)
$$

In [51]:
def cot(theta):
    return 1 / np.tan(theta)

def cot_difference(x, y):
    left_side = cot(x) - cot(x + y)
    right_side = (cot(x + y) * cot(x) + 1) * np.tan(y)
    return left_side, right_side

Some tests:

In [52]:
cot_difference(1.0, 1e-20)

(0.0, 1.4122829274373917e-20)

In [48]:
cotdiff(big(1.0), big(1e-20)) # compute in BigFloat precision

NameError: name 'big' is not defined

# Problem 3: Newtonish methods

Suppose we are trying to find a root $f(x)=0$ and we are given ways to compute $f$, $f'$, and $f''$.   You will design an algorithm to take advantage of this, and apply it to $f(x)=x^3-1$.

(You should be able to base your code on the Newton square-root example from class.)

In [95]:
def f(x):
    return x**3 - 1

def df(x):
    return 3*x**2 

def ddf(x):
    return 6*x 

def root_newton(x0,tol=1e-24):
    root = x0
    iter = 0 
    while np.abs(f(root)) >= tol:
        iter +=1
        root = root - f(root)/df(root)
        print(root-1)
    print("iter",iter)
    return root 

def root_newton_second(x0,tol=1e-24):
    root = x0
    iter = 0 
    while np.abs(f(root)) >= tol:
        iter +=1
        discriminant = (df(root)**2 - 2 * f(root) * ddf(root))
        if discriminant < 0:
            root = root - f(root)/df(root)
        else:
            root = root - 2 * f(root) / (df(root) + np.sign(df(root)) * discriminant**(1/2))
        print(root-1)

    print("iter",iter)
    return root 




In [96]:
r_1 = root_newton(2)

r_2 = root_newton_second(2)

print(r_1,r_2)


0.4166666666666665
0.1105344098423684
0.010636768404556296
0.0001115573039491835
1.2443181152121952e-08
2.220446049250313e-16
0.0
iter 7
0.4166666666666665
-0.030805118567510337
9.753390485256475e-06
-3.3306690738754696e-16
0.0
iter 5
1.0 1.0


# Problem 4: Addition, another way

In [ ]:
def div2sum(x,f=1,last=len(x)):
    n = last - f + 1 
    if n < 2:
        s = np.zeros(x)
        for i in range(f,last+1):
            s[i] += x[i]
        return s 
    else:
        mid = (f+last)//2
        return div2sum(x,f,mid) + div2sum(x,mid+1,last)


In [ ]:
# Sum x[first:last].  This function works, but is a little slower than we would like.
function div2sum(x, first=1, last=length(x))
    n = last - first + 1;
    if n < 2
        s = zero(eltype(x))
        for i = first:last
            s += x[i]
        end
        return s
    else
        mid = div(first + last, 2) # find middle as (first+last)/2, rounding down
        return div2sum(x, first, mid) + div2sum(x, mid+1, last)
    end
end

# check its accuracy for a set logarithmically spaced n's.  Since div2sum is slow,
# we won't go to very large n or use too many points
N = round.(Int, 10 .^ range(1,7,length=50)) # 50 points from 10¹ to 10⁷
err = Float64[]
for n in N
    x = rand(Float32, n)
    xdouble = Float64.(x)
    push!(err, abs(div2sum(x) - sum(xdouble)) / abs(sum(xdouble)))
end

using PyPlot
loglog(N, err, "bo-")
title("simple div2sum")
xlabel("number of summands")
ylabel("relative error")
grid()

Time it vs. the built-in `sum` (which is also written in Julia):

In [ ]:
x = rand(Float32, 10^7)
@time div2sum(x)
@time div2sum(x)
@time div2sum(x)
@time sum(x)
@time sum(x)
@time sum(x)

You should notice that it's pretty darn slow compared to `sum`, although in an absolute sense it is pretty good.  Make it faster: